# The analysis of multiscale model results: a list of results

This notebook shows various analyses of the results of the multiscale model. We go into the details of the pulse shaping, the plasma profile, and we show also the build up of the harmonic signal.

See complementary analyses:
* [Jupyter notebook processing a list of results.](./analyse_cell_list.ipynb)
* [Jupyter notebook analysing a file, which contains also the outputs of TDSE module](../density_profile/analyse_density_profile.ipynb) (these results were removed in the actual tutorial because they are the largest datasets from the model).



## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
from contextlib import ExitStack
import os
import h5py
import sys
import copy
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG

anims_author = 'Jan Vábek'

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

Here we dicide if we `show` animations here or `save` them as well.

In [ ]:
visualisation = 'show'

## Load data

Compared to the [analysis of a single file](./analyse_cell_single.ipynb), we reduce the resolution in $\rho$ to reduce the size of data within this notebook.

In [ ]:
# here we specify a coarser r-grid while loading the data
r_resolution_specifier = [False, 10e-6/3., 220e-6] # ['use the coarser data', radial step, maximal radial coordinate] (all in SI units)

demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'gas_cell')



h5files = [os.path.join(demos_path, foo) for foo in 
            ['results_cell_1.h5','results_cell_2.h5']] # list of loaded simulations

# with h5py.File(h5file1,'r') as f:

#     # load cuprad data = pulse propagation
#     CUPRAD_res = dfC.get_data(f)
#     CUPRAD_res.get_plasma(f)

# load the data
Nsim = len(h5files)
CUPRAD_res = []
with ExitStack() as stack:
    # Open all files and store file objects in a list
    files = [stack.enter_context(h5py.File(h5file, 'r')) for h5file in h5files]
    
    # Load data from each file and append it to CUPRAD_res
    for f in files:
        CUPRAD_res.append(dfC.get_data(f,r_resolution=r_resolution_specifier))
        CUPRAD_res[-1].get_plasma(f,r_resolution=r_resolution_specifier)

zmax_all = np.min([foo.zgrid[-1] for foo in CUPRAD_res])

Here we print some basic characteristics of the simulation. We assume that the different simulations differs only in the discretisation in $z$ due to adaptive steps and all the other parameters are the same.

In [ ]:
# code to generate the following text
display(Markdown(
rf"""* The ($1/\mathrm{{e}}$) entry pulse duration ${
      CUPRAD_res[0].pulse_duration_entry   
      :.1f}~\mathrm{{fs}}$
* The box size is ($z-$ and $\rho$-grids start at 0):
    * $z_{{\mathrm{{max}}}}={
      1e3*zmax_all
      :.1f}~\mathrm{{mm}},~N_{{z,\text{{min}}}}={
      np.min([np.shape(foo.E_zrt)[0] for foo in CUPRAD_res])   
      :.0f},~N_{{z,\text{{max}}}}={
      np.max([np.shape(foo.E_zrt)[0] for foo in CUPRAD_res])   
      :.0f}$,
    * $t_{{\mathrm{{min/max}}}}=\mp{
      1e15*CUPRAD_res[0].tgrid[-1]   
      :.0f}~\mathrm{{fs}},~N_t={
      np.shape(CUPRAD_res[0].E_zrt)[2]   
      :.0f}$,
    * $\rho_{{\mathrm{{max}}}}={
      1e6*CUPRAD_res[0].rgrid[-1]   
      :.0f}~\mu\mathrm{{m}},~N_\rho={
      np.shape(CUPRAD_res[0].E_zrt)[1]   
      :.0f}.$
"""))

## Plot the propagating pulse
The visualisation is analogical to [the single-file case](./analyse_cell_single.ipynb), the difference is that the $z$-grids are different for each file. We thus change the approach and define first the $z$-values for plotting and find corresponiding subarrays. Next, we show only the electric fields here.

In [ ]:
tlim = np.asarray((-60,60))   # [fs]  the time range for plotting the propagating pulse
rlim = 300                    # [mum] the radial range for plotting the propagating pulse

dz_plot = 20e-6 # [m]
N_columns = 2 # this specifies into how many columns is the figure split

save_animation = (visualisation == 'save')
if save_animation:
    ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'gas_cell', 'export')
    if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)

In [ ]:
# Code to generate the animated figure

Nz_plot = round(zmax_all/dz_plot)
k_t_min, k_t_max = tuple(zip(*[mn.FindInterval(1e15*CUPRAD_res[k1].tgrid,1.05*tlim) for k1 in range(len(CUPRAD_res))]))


Nrows = Nsim//N_columns if (Nsim%N_columns==0) else (Nsim//N_columns)+1

fig, axes = plt.subplots(Nrows, N_columns, figsize=(15, Nrows*4))  # Create a 2x2 grid of subplots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):
    
    # data_orig = 1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]]
    # rgrid_orig = 1e6*CUPRAD_res[k1].rgrid

    # data_totr = np.transpose(data_orig)

    rgrid_sym, data_sym = mn.symmetrize_y(1e6*CUPRAD_res[k1].rgrid, 1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]].T)

    pc = ax.pcolormesh(1e15*CUPRAD_res[k1].tgrid[k_t_min[k1]:k_t_max[k1]],
                       rgrid_sym,
                       data_sym.T,
                       shading='auto', cmap='seismic')

    # pc = ax.pcolormesh(1e15*CUPRAD_res[k1].tgrid[k_t_min[k1]:k_t_max[k1]],
    #                    1e6*CUPRAD_res[k1].rgrid,
    #                    1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]],
    #                    shading='auto', cmap='seismic')
    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.set_ylabel(r'$\mathcal{E}$ [GV/m]', rotation=90)
    cbars.append(cbar)
    

    # Set axis properties
    ax.set_xlim(tlim)
    ax.set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[0]))
    ax.set_xlabel(r'$t~[\mathrm{fs}]$')
    ax.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

# for k1 in range(Nrows): axes[Nrows*k1].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')  # Set ylabel only for the left plots
for k1 in range(Nsim,N_columns*Nrows): axes[k1].axis('off')

# Create colorbars using list comprehensions


def update(frame):
    for k1 in range(len(CUPRAD_res)):
        kz_local = mn.FindInterval(CUPRAD_res[k1].zgrid,frame*dz_plot)
        # Update the data for each subplot
        data = 1e-9*CUPRAD_res[k1].E_zrt[kz_local, :, k_t_min[k1]:k_t_max[k1]]

        data_sym = mn.symmetrize_y(1e6*CUPRAD_res[k1].rgrid, 1e-9*CUPRAD_res[k1].E_zrt[kz_local, :, k_t_min[k1]:k_t_max[k1]].T)[1].T
        
        # Update the colors
        pcs[k1].set_array(data_sym.ravel())
        max_value_symmetric = np.max(np.abs((data.min(), data.max())))
        pcs[k1].set_clim(-max_value_symmetric,max_value_symmetric)
        cbars[k1].update_normal(pcs[k1])

        axes[k1].set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[kz_local]))

    return pcs

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 
# plt.show()

ani = matplotlib.animation.FuncAnimation(fig,
                                         update,
                                         frames=Nz_plot,
                                         blit=True)

if save_animation:
    # Define the writer using ffmpeg for mp4 format and save it
    FFmpegWriter = matplotlib.animation.writers['ffmpeg']
    writer = FFmpegWriter(fps=fps, metadata=dict(artist=animation_author), bitrate=1800)

    ani.save(os.path.join(ani_outpath,'gas_cell_list_Efield.mp4'), writer=writer)

HTML(ani.to_jshtml())


## Plasma channel
Here we show plasma channels after the passage of the pulse.

In [ ]:
# Code to generate the figure

Nrows = Nsim//N_columns if (Nsim%N_columns==0) else (Nsim//N_columns)+1

fig, axes = plt.subplots(Nrows, N_columns, figsize=(15, Nrows*4))  # Create a 2x2 grid of subplots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):
    
    # data_orig = 1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]]
    # rgrid_orig = 1e6*CUPRAD_res[k1].rgrid

    # data_totr = np.transpose(data_orig)

    rgrid_sym, data_sym = mn.symmetrize_y(1e6*CUPRAD_res[k1].plasma.rgrid, (1e2/CUPRAD_res[k1].effective_neutral_particle_density)*CUPRAD_res[k1].plasma.value_zrt[:, :, -1])

    pc = ax.pcolormesh(1e3*CUPRAD_res[k1].plasma.zgrid,
                       rgrid_sym,
                       data_sym.T,
                       shading='auto')

    # pc = ax.pcolormesh(1e15*CUPRAD_res[k1].tgrid[k_t_min[k1]:k_t_max[k1]],
    #                    1e6*CUPRAD_res[k1].rgrid,
    #                    1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]],
    #                    shading='auto', cmap='seismic')
    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.set_ylabel(r'relative plasma density [%]', rotation=90)
    cbars.append(cbar)
    

    # Set axis properties
    ax.set_title(f"simulation {k1+1}")
    ax.set_xlabel(r'$z~[\mathrm{mm}]$')
    ax.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

# for k1 in range(Nrows): axes[Nrows*k1].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')  # Set ylabel only for the left plots
for k1 in range(Nsim,N_columns*Nrows): axes[k1].axis('off')

# Create colorbars using list comprehensions



# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 
plt.show()

## XUV camera

Finally, we show the far field harmonic spectra and cummulative signal in the medium

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec

# Set parameters
Nsim = len(CUPRAD_res)  # Number of simulations
N_columns = 2  # Number of columns in the figure

Nrows = Nsim // N_columns if (Nsim % N_columns == 0) else (Nsim // N_columns) + 1

# Set up a grid with subplots, using nested gridspec for each element
fig = plt.figure(figsize=(15, Nrows * 8))
gs_outer = gridspec.GridSpec(Nrows, N_columns, figure=fig, wspace=0.4, hspace=0.6)

pcs = []
cbars = []

# Create main plots and bar plots using nested grids
for k1 in range(Nsim):
    # Create nested gridspec for each subplot
    gs_inner = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs_outer[k1], height_ratios=[3, 1], hspace=0.3)
    
    # Create axes for the main plot
    ax_main = fig.add_subplot(gs_inner[0])
    
    # Symmetrize data
    rgrid_sym, data_sym = mn.symmetrize_y(1e6 * CUPRAD_res[k1].plasma.rgrid, 
                                          (1e2 / CUPRAD_res[k1].effective_neutral_particle_density) * CUPRAD_res[k1].plasma.value_zrt[:, :, -1])

    # Create the main plot (pcolormesh)
    pc = ax_main.pcolormesh(1e3 * CUPRAD_res[k1].plasma.zgrid,
                            rgrid_sym,
                            data_sym.T,
                            shading='auto')

    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax_main)
    cbar.ax.set_ylabel(r'relative plasma density [%]', rotation=90)
    cbars.append(cbar)
    
    # Set axis properties for the main plot
    ax_main.set_title(f"Plot {k1 + 1}")
    ax_main.set_xlabel(r'$z~[\mathrm{mm}]$')
    ax_main.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')
    
    # Create axes for the bar plot below the main plot
    ax_bar = fig.add_subplot(gs_inner[1])
    
    # Create a bar plot of x^n (example with n=2)
    x = np.arange(1, 6)
    n = 2
    y = x ** n
    ax_bar.bar(x, y, color='grey')
    
    # Set axis properties for the bar plot
    ax_bar.set_xlabel(r'$x$')
    ax_bar.set_ylabel(r'$x^n$')
    ax_bar.set_title(r'$x^n$ with $n=2$', fontsize=10)
    ax_bar.set_xticks(x)

# Turn off any unused axes if the number of subplots is less than rows * columns
for k1 in range(Nsim, N_columns * Nrows):
    ax = fig.add_subplot(gs_outer[k1])
    ax.axis('off')

# Ensure the layout does not have overlaps and everything is nicely spaced
# fig.tight_layout()
plt.show()


In [ ]:
# Code to generate the figure

Nrows = Nsim//N_columns if (Nsim%N_columns==0) else (Nsim//N_columns)+1

fig, axes = plt.subplots(Nrows, N_columns, figsize=(15, Nrows*4))  # Create a 2x2 grid of subplots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):
    
    # data_orig = 1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]]
    # rgrid_orig = 1e6*CUPRAD_res[k1].rgrid

    # data_totr = np.transpose(data_orig)

    rgrid_sym, data_sym = mn.symmetrize_y(1e6*CUPRAD_res[k1].plasma.rgrid, (1e2/CUPRAD_res[k1].effective_neutral_particle_density)*CUPRAD_res[k1].plasma.value_zrt[:, :, -1])

    pc = ax.pcolormesh(1e3*CUPRAD_res[k1].plasma.zgrid,
                       rgrid_sym,
                       data_sym.T,
                       shading='auto')

    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.set_ylabel(r'relative plasma density [%]', rotation=90)
    cbars.append(cbar)
    

    # Set axis properties
    ax.set_title("xxx")
    ax.set_xlabel(r'$z~[\mathrm{mm}]$')
    ax.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')


for k1 in range(Nsim,N_columns*Nrows): axes[k1].axis('off')

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 
plt.show()

In [ ]:
rmax = 0.007                # [m]    the radial dimesion to read the data
XUV_theta_range = [-4, 4]   # [mrad] the divergence angle for plotting 
orders_to_plot = 4          # the range of the logarithmic plot of the spatially resolved harmonic spectra
# Hmax_plot_linear = 50       # the maximal frequency in the linear plot of the spatially resolved harmonic spectra


kz_step = 10    #       the step in $z$ for plotting (derived from the spacing used in the computational grid)

H_interest = np.asarray([19, 25, 37, 49])   # harmonics for which we show the build-up
multipliers = [1,4,10,150]                  # multipliers applied to the build-up to fit the figure nicely
delta_H = 1.                                # the camera spectral range to analyse signal of the harmoonics of the interest

In [ ]:
# import data & basic analyses
with h5py.File(h5file1,'r') as f1:
    # load Hankel data = XUV camera
    ogrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/ogrid'][:]
    rgrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/rgrid'][:]
    zgrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/zgrid'][0:-1:kz_step]

    camera_distance = mn.readscalardataset(f1,MMA.paths['Hankel_inputs']+'/distance_FF','N')
    theta_grid_Hankel = np.arctan(rgrid_Hankel/camera_distance)  # recompute the radial grid to the divergence

    Hgrid_Hankel = ogrid_Hankel/CUPRAD_res.omega0

    kr_max = mn.FindInterval(rgrid_Hankel, rmax) + 1
    rgrid_Hankel_full = copy.copy(rgrid_Hankel)
    rgrid_Hankel = rgrid_Hankel[:kr_max]
    theta_grid_Hankel = theta_grid_Hankel[:kr_max]


    cumulative_field =    f1[MMA.paths['Hankel_outputs']+'/cumulative_field'][0:-1:kz_step,:kr_max,:,0] +\
                    1j*f1[MMA.paths['Hankel_outputs']+'/cumulative_field'][0:-1:kz_step,:kr_max,:,1] 
    
    entry_plane_transform = f1[MMA.paths['Hankel_outputs']+'/entry_plane_transform'][:,:,0] +\
                    1j*f1[MMA.paths['Hankel_outputs']+'/entry_plane_transform'][:,:,1]

    exit_plane_transform = f1[MMA.paths['Hankel_outputs']+'/exit_plane_transform'][:,:,0] +\
                1j*f1[MMA.paths['Hankel_outputs']+'/exit_plane_transform'][:,:,1]

# find maxima of the harmonics of the interest
H_idx = [tuple(mn.FindInterval(Hgrid_Hankel,(H_interest[k1]-delta_H, H_interest[k1]+delta_H))) for k1 in range(len(H_interest))]
H_max_interest = [np.max(np.abs(cumulative_field[:,:,H_idx[k1][0]:H_idx[k1][1]]),axis=(1,2)) for k1 in range(len(H_interest))]    

In [ ]:
# Code to create the following animated figure
fig = plt.figure(figsize=(14, 6))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((3, 2), (0, 0), rowspan=2)  # Upper left
ax2 = plt.subplot2grid((3, 2), (0, 1), rowspan=2)  # Upper right
ax3 = plt.subplot2grid((3, 2), (2, 0), colspan=2)  # Bottom, spanning both columns

theta_grid_Hankel = theta_grid_Hankel[:kr_max]
r_grid, sym_data = mn.symmetrize_y(rgrid_Hankel, np.abs(cumulative_field[0,:,:]).T)
theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(cumulative_field[0,:,:]).T)



pc1 = ax1.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto')
pc2 = ax2.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))

ax1.set_ylim(XUV_theta_range)
ax2.set_ylim(XUV_theta_range)

# ax1.set_xlim(Hgrid_Hankel[0],Hmax_plot_linear)

ax1.set_title('spatially resolved XUV spectrum (linscale)')
ax2.set_title('spatially resolved XUV spectrum (logscale)')
ax1.set_ylabel(r'divergence [mrad]')
for cbar in (cbar1, cbar2): cbar.ax.set_xlabel(r'$|\mathcal{E}_{XUV}|$ [arb.u.]')
ax1.set_ylabel(r'divergence [mrad]')
H_min, H_max = ax1.get_xlim() 
odd_ticks_major = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 10) # Generate odd ticks within the H-range
odd_ticks_minor = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 2)  # Generate odd ticks within the H-range
for ax in (ax1, ax2):
    ax.set_xlabel('harmonic order [-]')
    ax.set_xticks(odd_ticks_major)
    ax.set_xticks(odd_ticks_minor, minor = True)


cbar1 = fig.colorbar(pc1, ax=ax1)
cbar2 = fig.colorbar(pc2, ax=ax2) #, orientation='horizontal')
cbar2.ax.set_ylabel(r'$|\mathcal{E}_{XUV}|$ [arb.u.]', rotation=90)

# plot lines at selected harmonic orders
for k1 in range(len(H_interest)):
    ax1.plot(2*[H_interest[k1]],
             1e3*theta_grid_sym[-1]*np.asarray([-1,1]),
             'w:',alpha = 0.4)
    ax2.plot(2*[H_interest[k1]],
             1e3*theta_grid_sym[-1]*np.asarray([-1,1]),
             'w:',alpha = 0.4)



title = fig.suptitle("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')


# ax3.plot(1e3*CUPRAD_res.density_mod_zgrid,
#             CUPRAD_res.density_mod_profile_mbar,'gray')
ax3.set_xlabel(r'$z~[\mathrm{mm}]$')
# ax3.set_ylabel(r'$p~[\mathrm{mbar}]$')

progress_line, = ax3.plot([], [], 'r-')  

# ax3.fill_between(1e3*CUPRAD_res.density_mod_zgrid, CUPRAD_res.density_mod_profile_mbar,color='lightgrey')

ax3.set_xlim((1e3*CUPRAD_res.zgrid[0],1e3*CUPRAD_res.zgrid[-1]))

# ax3_tw = ax3.twinx()

# normalised signals
max_signal = np.max([signal for signal in H_max_interest])
# multipliers = [max_signal/np.max(signal) for signal in H_max_interest]

for k1 in range(len(H_interest)):
    if (len(zgrid_Hankel) == len(H_max_interest[k1][:])): signal_plot = H_max_interest[k1][:]
    else: signal_plot = np.append(0,H_max_interest[k1][:])
    ax3.plot(1e3*zgrid_Hankel,multipliers[k1]*signal_plot, label='H'+str(H_interest[k1])+f' (x {multipliers[k1]:.1f})')

ax3.set_ylabel(r'XUV signal $[\mathrm{arb. u.}]$')

# ax3_tw.plot(1e3*zgrid_Hankel,np.append(0,H_max_interest[0][:]), 'b--', label='xxx')

ax3.legend()

def update(frame):
    # Update the data
    data = (mn.symmetrize_y(rgrid_Hankel, np.abs(cumulative_field[frame,:,:]).T)[1]).T

    pc1.set_array(data.ravel())
    pc1.set_clim(data.min(), data.max())

    # Update the colors
    pc2.set_array(data.ravel())
    # pc.set_clim(data.min(), data.max())
    pc2.set_clim((10**(-orders_to_plot))*data.max(), data.max())
    # cbar.update_normal(pc2)

    title.set_text("z={:.2f}".format(1e3*zgrid_Hankel[frame+1]) + ' mm')

    # Update the progress indicator
    progress_line.set_data([1e3*zgrid_Hankel[frame+1], 1e3*zgrid_Hankel[frame+1]],
                             ax3.get_ylim())

    return [pc1,pc2, progress_line]


# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout()

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(zgrid_Hankel)-1, blit=True)
# ani = matplotlib.animation.FuncAnimation(fig, update, frames=3, blit=True) # for testing the plot


if visualisation == 'save':
    # Define the writer using ffmpeg for mp4 format and save it
    Writer = matplotlib.animation.writers['ffmpeg']
    writer = Writer(fps=3, metadata=dict(artist=anims_author), bitrate=1800)

    ani.save(os.path.join(ani_outpath,'gas_cell_single_spectrum.mp4'), writer=writer)

HTML(ani.to_jshtml())


The output data also contain the Hankel transforms of the sources at the entry and exit plane of the medium.

In [ ]:
# Code to generate the figures of the Hankel transforms
logscale = True
normalise_spectra = True
orders_to_plot = 3 




fig = plt.figure(figsize=(17, 7))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right

theta_grid_Hankel = np.arctan(rgrid_Hankel_full/camera_distance)  # recompute the radial grid to the divergence
theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(entry_plane_transform).T/np.max(np.abs(entry_plane_transform)) if normalise_spectra
                                                              else np.abs(entry_plane_transform).T)
scale_kwargs = {'norm' : colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max())} if logscale else {}
pc1 = ax1.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',**scale_kwargs)
cbar1 = fig.colorbar(pc1, ax=ax1, orientation = 'horizontal')


theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(exit_plane_transform).T/np.max(np.abs(exit_plane_transform)) if normalise_spectra
                                                              else np.abs(exit_plane_transform).T)
scale_kwargs = {'norm' : colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max())} if logscale else {}
pc2 = ax2.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',**scale_kwargs)
cbar2 = fig.colorbar(pc2, ax=ax2, orientation = 'horizontal')

for cbar in (cbar1, cbar2): cbar.ax.set_xlabel(r'$|\mathcal{E}_{XUV}|$ [arb.u.]')
ax1.set_ylabel(r'divergence [mrad]')
H_min, H_max = ax1.get_xlim() 
odd_ticks_major = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 10) # Generate odd ticks within the H-range
odd_ticks_minor = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 2)  # Generate odd ticks within the H-range
for ax in (ax1, ax2):
    ax.set_xlabel('harmonic order [-]')
    ax.set_xticks(odd_ticks_major)
    ax.set_xticks(odd_ticks_minor, minor = True)

plt.show()